In [7]:
import pandas as pd
import numpy as np
import altair as alt

alt.data_transformers.enable('data_server')

# Wrangling

whitewine = pd.read_csv('Data/raw/winequality-red.csv', sep=';')
redwine = pd.read_csv('WineVison/Data/raw/winequality-red.csv', sep=';')

whitewine["type"] = "white"
redwine["type"] = "red"

wine = redwine.append(whitewine)
wine

FileNotFoundError: [Errno 2] File WineVison/Data/raw/winequality-red.csv does not exist: 'WineVison/Data/raw/winequality-red.csv'

In [52]:
# Correlation Graph

# Get correlations for each wine type
corr_df_white = wine.loc[wine['type'] == 'white'].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
corr_df_white["type"] = "white"

corr_df_red = wine.loc[wine['type'] == 'red'].select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
corr_df_red["type"] = "red"

# Bind them together
corr_df = corr_df_white.append(corr_df_red)

corr_df.loc[corr_df['corr'] == 1, 'corr'] = 0 #Remove full correlations on diag
corr_df['abs'] = corr_df['corr'].abs()


# Make chart
click = alt.selection_multi(fields=['type'], bind='legend') 
alt.Chart(corr_df,title="Correlation Plot for Numeric Features").mark_square().encode(
    color=alt.Color('type', scale=alt.Scale(domain=['red', 'white'],
            range=['darkred', 'blue'])),
    x='level_0',
    y='level_1',
    size='abs',
    opacity=alt.condition(click, alt.value(0.7), alt.value(0)),
    tooltip=["type", "corr"]
).configure_title(fontSize=18).properties(height=250, width=250).add_selection(click)

alt.Chart(...)

In [65]:
# Scatterplot
click = alt.selection_multi(fields=['type'], bind='legend')
chart = alt.Chart(wine).mark_point().encode(
    alt.X("alcohol"),
    alt.Y("sulphates"),
    alt.Color("type"),
    opacity = alt.condition(click, alt.value(0.7), alt.value(0))
)

regression = chart.transform_regression('alcohol','sulphates', groupby = ["type"],
                                        # By default lines don't go beyond data and are hard to read in this dense dataset
                                       extent = [min(wine['alcohol']) - 1, max(wine['alcohol']) + 1]).mark_line(size = 5)

(chart + regression).add_selection(click)

alt.LayerChart(...)

In [54]:
corr_df["level_0"].unique()

array(['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'], dtype=object)